<a href="https://colab.research.google.com/github/RomaTruba/dzSEM/blob/main/S_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [2]:

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:

documents = [
    "Собаки — домашние животные, средний вес 7-30 кг",
    "Собаки часто используются как охранники, вес 10-30 кг",
    "Nissan Leaf II— электромобиль с запасом хода 550 км",
    "Бананы содержат витамин А и полезны для здоровья",
    "Грейпфруты богаты витаминами и растут в теплом климате"
]

doc_embeddings = model.encode(documents)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

def compare_metrics(query, top_k=2):
    query_embed = model.encode([query])

    # Расчет разных метрик
    metrics = {
        "cosine": cosine_similarity(query_embed, doc_embeddings)[0],
        "euclidean": 1 / (1 + euclidean_distances(query_embed, doc_embeddings)[0]),
        "manhattan": 1 / (1 + manhattan_distances(query_embed, doc_embeddings)[0])
    }

    # Вывод результатов
    for name, scores in metrics.items():
        print(f"\nМетрика: {name.upper()}")
        results = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)[:top_k]
        for doc, score in results:
            print(f"{score:.4f}: {doc[:50]}...")

# Пример сравнения
compare_metrics("Какие фрукты полезны?")


Метрика: COSINE
0.6659: Собаки часто используются как охранники, вес 10-30...
0.6657: Грейпфруты богаты витаминами и растут в теплом кли...

Метрика: EUCLIDEAN
0.5502: Собаки часто используются как охранники, вес 10-30...
0.5502: Грейпфруты богаты витаминами и растут в теплом кли...

Метрика: MANHATTAN
0.0738: Собаки часто используются как охранники, вес 10-30...
0.0719: Грейпфруты богаты витаминами и растут в теплом кли...


In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.2 MB/s eta 0:00:00


In [7]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553251 sha256=20c42dcdf03ca09a7af4e50b7e235a3227ece122d1cfc4109816d55ad8883251
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [8]:
import faiss
from annoy import AnnoyIndex
import time

# FAISS индекс
d = doc_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(d)
faiss.normalize_L2(doc_embeddings)
faiss_index.add(doc_embeddings.astype('float32'))

# Annoy индекс
annoy_index = AnnoyIndex(d, 'angular')
for i, vec in enumerate(doc_embeddings):
    annoy_index.add_item(i, vec)
annoy_index.build(10)

# Тест производительности
def test_speed(query, repeats=1000):
    query_embed = model.encode([query]).astype('float32')[0]

    # FAISS
    start = time.time()
    for _ in range(repeats):
        faiss_index.search(query_embed.reshape(1, -1), 2)
    faiss_time = (time.time() - start)/repeats

    # Annoy
    start = time.time()
    for _ in range(repeats):
        annoy_index.get_nns_by_vector(query_embed, 2, include_distances=True)
    annoy_time = (time.time() - start)/repeats

    print(f"FAISS: {faiss_time:.6f} сек/запрос")
    print(f"Annoy: {annoy_time:.6f} сек/запрос")

test_speed("Домашние животные")

FAISS: 0.000003 сек/запрос
Annoy: 0.000039 сек/запрос


In [9]:
from sklearn.metrics import precision_recall_fscore_support


test_queries = {
    "Животные": [0, 1],
    "Фрукты": [3, 4],
    "Автомобили": [2]
}

def evaluate():
    results = []
    for query, relevant in test_queries.items():
        # Поиск через FAISS
        emb = model.encode([query]).astype('float32')
        _, idxs = faiss_index.search(emb, 2)
        retrieved = idxs[0].tolist()

        # Расчет метрик
        y_true = [1 if i in relevant else 0 for i in range(len(documents))]
        y_pred = [1 if i in retrieved else 0 for i in range(len(documents))]

        p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
        results.append((query, p, r, f1))

    return results

# Вывод результатов
print("Оценка качества:")
for query, p, r, f1 in evaluate():
    print(f"{query:15} Precision: {p:.2f}, Recall: {r:.2f}, F1: {f1:.2f}")

Оценка качества:
Животные        Precision: 0.50, Recall: 0.50, F1: 0.50
Фрукты          Precision: 0.50, Recall: 0.50, F1: 0.50
Автомобили      Precision: 0.00, Recall: 0.00, F1: 0.00


In [11]:
!pip install gradio

In [12]:
import gradio as gr

def semantic_search(query):
    query_embed = model.encode([query]).astype('float32')
    distances, indices = faiss_index.search(query_embed, 3)

    results = []
    for i, idx in enumerate(indices[0]):
        results.append(f"{i+1}. {documents[idx]} (score: {distances[0][i]:.3f})")

    return "\n\n".join(results)

In [13]:
# Создание интерфейса
iface = gr.Interface(
    fn=semantic_search,
    inputs=gr.Textbox(label="Ваш запрос"),
    outputs=gr.Textbox(label="Результаты"),
    examples=[["Какие животные домашние?"], ["Что полезно для здоровья?"]]
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e47942200d0da0638.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
